In [16]:
import pandas as pd
import regex # trabalhar com expressões regulares
import nltk #caso não funcione, adicione a linha nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report
from goose3 import Goose #para baixar o texto da notícia


dados_noticias = pd.read_csv('News_corpus.csv')
len(dados_noticias.loc[6090,'preprocessed_news'])

2593

In [17]:
def remove_stopwords(texto):
    stops_list = stopwords.words("portuguese")
    word_tokens = word_tokenize(texto)
    texto_sem_stops = [w for w in word_tokens if w not in stops_list]
    return " ".join(texto_sem_stops)

In [ ]:
dados_noticias['preprocessed_news'] = dados_noticias['preprocessed_news'].apply(lambda x: regex.sub('[^\p{Latin}]', u' ', str(x)))
dados_noticias['preprocessed_news'] = dados_noticias['preprocessed_news'].apply(lambda x: str(x).lower())
dados_noticias['preprocessed_news'] = dados_noticias['preprocessed_news'].apply(remove_stopwords)
import spacy
nlp = spacy.load('pt_core_news_sm')

def spacy_lemmatizer(texto):
    word_tokens = nlp(texto)
    texto_lematizado = [w.lemma_ for w in word_tokens]
    return " ".join(texto_lematizado)

dados_noticias['preprocessed_news'] = dados_noticias['preprocessed_news'].apply(spacy_lemmatizer)

vectorizer = CountVectorizer()
BoW_matriz = vectorizer.fit_transform(dados_noticias['preprocessed_news'])
BoW_dataframe = pd.DataFrame(BoW_matriz.toarray(), columns=vectorizer.get_feature_names())
corpus_bow = BoW_dataframe
categorias = dados_noticias['label']

X_train, X_test, y_train, y_test = train_test_split(
                                        corpus_bow, 
                                        categorias,
                                        test_size=0.2)
naive_bayes = MultinomialNB()
naive_bayes.fit(X_train, y_train)
classificacoes = naive_bayes.predict(X_test)

print(classification_report(y_test, classificacoes))


cm = confusion_matrix(y_test, classificacoes)

sns.heatmap(cm, square=True, annot=True, cmap='RdBu', cbar=False,
            xticklabels=categorias.unique(),
            yticklabels=categorias.unique()
           )

plt.xlabel("Classe real")
plt.ylabel("Classe predita")
plt.show()
